In [6]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dags.module.object_client import MinioClient
from minio import Minio
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import json
import time
from lxml import html

FileNotFoundError: [WinError 3] The system cannot find the path specified: './yahoo_data'

In [8]:
url = f"https://finance.yahoo.com/quote/AADI/profile"
# Retry strategy
retries = Retry(total=10, backoff_factor=0.1)
adapter = HTTPAdapter(max_retries=retries)
# Session
session = requests.Session()
session.mount('http://', adapter=adapter)
session.mount('https://', adapter=adapter)
# Requests
r = session.get(
    url, headers={"user-agent": "Mozilla/5.0", "from": "youremail@gmail.com"}, stream=True)
# Parse HTML content using BS4
tree = html.fromstring(r.content)
# HTML content
data = tree.xpath('//*[@id="nimbus-app"]/section/section/section/article')
# Check if HTML content is empty
# html_content = str(html.tostring(data[0], pretty_print=True)).replace("b\'", "").replace(" \\n\'", "")
# html_content
data

[<Element article at 0x232262cdb20>]

In [7]:
def extract_HTML_content(**kwargs):
    # Arguments for the function
    symbol = kwargs.get("symbol")
    bucket_name = kwargs.get("bucket_name")
    bucket_name_2 = kwargs.get("bucket_name_2")
    
    # Filename
    filename = os.path.join(f"yahoo_data_info/{symbol}.html")
    # if os.path.exists(filename):
    #     return
    
    # URL
    url = f"https://finance.yahoo.com/quote/{symbol}/profile"

    # Retry strategy
    retries = Retry(total=10, backoff_factor=0.1)
    adapter = HTTPAdapter(max_retries=retries)
    
    # Session
    session = requests.Session()
    session.mount('http://', adapter=adapter)
    session.mount('https://', adapter=adapter)
    
    # Requests
    r = session.get(url, headers={"user-agent": "Mozilla/5.0", "from": "youremail@gmail.com"})
    
    # Parse HTML content using BS4
    soup = BeautifulSoup(r.text, "html.parser")
    
    # Minio client
    minio_client = MinioClient("localhost", "9000", access_key="minio", secret_key="minio123")
    
    # HTML content
    data = soup.find("article")
    
    # Check if HTML content is empty
    if data == None:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(str(data))
            print(f"File {filename} is empty. Put into the {bucket_name_2} bucket.")
        minio_client.put_object(bucket_name=bucket_name_2, obj_name=filename, obj_file=filename)
    else:
        # Write into a HTML file
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(str(data.prettify()))
            print(f"File {filename} written successfully. Put into the {bucket_name} bucket.")
        minio_client.put_object(bucket_name=bucket_name, obj_name=filename, obj_file=filename)

In [8]:
for symbol in list_symbols:
    extract_HTML_content(symbol=symbol, bucket_name="bronze", bucket_name_2="trashbin")
    time.sleep(10)

File yahoo_data_info/A.html written successfully. Put into the bronze bucket.
File yahoo_data_info/AA.html written successfully. Put into the bronze bucket.
File yahoo_data_info/AAA.html written successfully. Put into the bronze bucket.
File yahoo_data_info/AAAU.html written successfully. Put into the bronze bucket.
File yahoo_data_info/AACG.html written successfully. Put into the bronze bucket.
File yahoo_data_info/AACI.html written successfully. Put into the bronze bucket.
File yahoo_data_info/AACIU.html written successfully. Put into the bronze bucket.
File yahoo_data_info/AACIW.html written successfully. Put into the bronze bucket.
File yahoo_data_info/AACT.html written successfully. Put into the bronze bucket.
File yahoo_data_info/AADI.html written successfully. Put into the bronze bucket.
File yahoo_data_info/AADR.html written successfully. Put into the bronze bucket.
File yahoo_data_info/AAGR.html written successfully. Put into the bronze bucket.
File yahoo_data_info/AAGRW.html 

In [110]:
def transform_data_from_HTML(symbol, bucket_name_1, bucket_name_2):
    
    # Setup MinIO client
    minio_client = Minio("localhost:9000", "minio", "minio123", secure=False)
    
    # Extract HTML content from landing zone
    html_content = minio_client.get_object(bucket_name=bucket_name_1, object_name=f"yahoo_data_info/{symbol}.html").data.decode('utf-8')
    
    # Setup BS4
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Filename of object
    filename = f"yahoo_data_info/{symbol}.json"
    
    # Profile type variable
    type_text = soup.find("article").\
        find("section", "yf-16025kh reverseColumn twothirds").\
        find("section", "yf-1hj9jti").\
        find("header", "medium mb-4 yf-1trny4b font-condensed").\
        find("h3")
    type = type_text.text.replace(' ', '')
    
    # Data extraction holder
    result = dict()
    
    # Corpo stock
    if type == "KeyExecutives":
        description = soup.find("article").\
            find("section", "yf-16025kh reverseColumn twothirds").\
            find("section", "yf-1hj9jti").\
            find("div").text.replace('\n', ' ').replace('                 ', '').replace('             ', '')
        
        # Check if the profile is empty
        if description == None:
            result = {
                    "Sector": "NaN",
                    "Industry": "NaN",
                    "Description": "NaN"
                }
            
        # Profile not empty, proceed to extract data from elements
        else:
            content = soup.find("article").\
                    find("section", "yf-16025kh reverseColumn twothirds").\
                    find_all("section")[1].find("dl").\
                    find_all("div")
            sector = content[0].find("dd").find("a").text.replace(' ', '').replace('\n', '')
            industry = content[1].find("a").text.replace(' ', '').replace('\n', '')
            
            # Data into a dictionary
            result = {
                "Sector": sector,
                "Industry": industry,
                "Description": description
            }
            
    # ETF stock
    else:
        summary = soup.find("article").\
            find("section", class_="yf-16025kh").\
            find_all("section")[0].find("p").text
        
        overview = soup.find("article").\
            find("section", class_="yf-16025kh").\
            find_all("section")[1].find("div").\
            find("table").find("tbody").find_all("tr")
            
        category = overview[0].find_all("td")[1].text
        fund_family = overview[1].find_all("td")[1].text
        net_assets = overview[2].find_all("td")[1].text
        legal_type = overview[5].find_all("td")[1].text
        
        result = {
            "Category": category,
            "Fund Family": fund_family,
            "Net Assets": net_assets,
            "Legal Type": legal_type,
            "Summary": summary
        }
    
    # Write data into a JSON file into MinIO staging zone
    with open(filename, 'w') as f:
        json.dump(result, f, indent=4, ensure_ascii=False)
    minio_client.fput_object(bucket_name=bucket_name_2, object_name=filename, file_path=filename)

In [111]:
transform_data_from_HTML("AAPL", "bronze", "silver")

In [4]:
import os
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from minio import Minio
from dotenv import dotenv_values
import json
from dags.module.object_client import MinioClient
from lxml import html

In [7]:
config = dotenv_values("dags/.env")
access_key = config['MINIO_ROOT_USER']
secret_key = config['MINIO_ROOT_PASSWORD']

In [33]:
html_content = minio_client.get_object(bucket_name="bronze", obj_name="yahoo_data_info/A.html", prefix="yahoo_data_info/").data.decode('utf-8')
tree = html.fromstring(html_content)

In [34]:
type = tree.xpath("//article/section[2]/section[1]/header[1]/h3[1]/text()")[0].replace('\n', '').replace(' ', '')
type

'KeyExecutives'

In [10]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="userdb",
    user="username",
    password="password",
    port=5439
)

cursor = conn.cursor()

In [1]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="userdb",
    user="username",
    password="password",
    port=5439
)

cursor = conn.cursor()

# Fact table queries

fact_table_sql = '''
    CREATE TABLE IF NOT EXISTS fact_stock_data(
        id SERIAL PRIMARY KEY,
        symbol VARCHAR(30) NOT NULL UNIQUE,
        info_id VARCHAR(30) NOT NULL,
        type VARCHAR(30) NOT NULL,
        description VARCHAR(1000),
        status VARCHAR(30)
    )
'''

# Dim tables

# dim_info_corpo_sql = '''
#     CREATE TABLE IF NOT EXISTS dim_info_corpo(
#         info_id VARCHAR(30) NOT NULL PRIMARY KEY,
#         sector VARCHAR(50),
#         industry VARCHAR(50)
#     );
# '''

dim_info_stock_sql = '''
    CREATE TABLE IF NOT EXISTS dim_info_stock(
        info_id VARCHAR(20) NOT NULL PRIMARY KEY,
        category VARCHAR(30),
        fund_family VARCHAR(30),
        net_assets VARCHAR(10),
        legal_type VARCHAR(30),
        sector VARCHAR(50),
        industry VARCHAR(50)
    );
'''

dim_hist_data_sql = '''
    CREATE TABLE IF NOT EXISTS dim_hist_data(
        record_id SERIAL,
        "Date" DATE NOT NULL,
        "Open" DECIMAL(8,1),
        "High" DECIMAL(8,1),
        "Low" DECIMAL(8,1),
        "Close" DECIMAL(8,1),
        "Adj Close" DECIMAL(8,1),
        "Volume" BIGINT,
        "Symbol" VARCHAR(30) NOT NULL,
        PRIMARY KEY ("Symbol", "Date")
    );
'''

fk_setup_sql = '''
    ALTER TABLE dim_hist_data ADD CONSTRAINT fk_fact_stock_data FOREIGN KEY ("Symbol") REFERENCES fact_stock_data(symbol);
    ALTER TABLE fact_stock_data ADD CONSTRAINT fk_dim_info_stock FOREIGN KEY (info_id) REFERENCES dim_info_stock(info_id);
'''

cursor.execute(fact_table_sql)
cursor.execute(dim_info_stock_sql)
cursor.execute(dim_hist_data_sql)
cursor.execute(fk_setup_sql)

index_sql = """
    CREATE UNIQUE INDEX index_stock_symbol on fact_stock_data(symbol);
    CREATE UNIQUE INDEX index_stock_info_id on fact_stock_data(info_id);  
"""

cursor.execute(f'''
                CREATE TABLE IF NOT EXISTS temp_dim_hist_data(
                record_id SERIAL,
                "Date" DATE NOT NULL,
                "Open" DECIMAL(8,1),
                "High" DECIMAL(8,1),
                "Low" DECIMAL(8,1),
                "Close" DECIMAL(8,1),
                "Adj Close" DECIMAL(8,1),
                "Volume" BIGINT,
                "Symbol" VARCHAR(30) NOT NULL,
                PRIMARY KEY ("Symbol", "Date")
                );
                ''')
cursor.execute(index_sql)
conn.commit()

In [6]:
cursor.execute('''
                TRUNCATE TABLE dim_info_stock;
                TRUNCATE TABLE fact_stock_data;
                TRUNCATE TABLE temp_dim_hist_data;
                TRUNCATE TABLE dim_hist_data;
                ''')
conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [14]:
import requests
from datetime import datetime, timezone, timedelta
midnight = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)
a_day_ago = midnight - timedelta(days=1)
current_day = int(midnight.timestamp())
a_day_ago = int(a_day_ago.timestamp())
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
url = lambda x: f"https://query{x}.finance.yahoo.com/v7/finance/download/AAPL?period1={a_day_ago}&period2={current_day}&interval=1d&events=history&includeAdjustedClose=true"
response = requests.get(url(1), headers=headers).content.decode('utf-8').split("\n")[1].split(",")
response

['2024-08-07',
 '206.899994',
 '213.639999',
 '206.389999',
 '209.820007',
 '209.820007',
 '63402300']

In [2]:
from dags.module.object_client import MinioClient
import io
import csv
from dotenv import dotenv_values
from datetime import datetime

config = dotenv_values("dags/.env")
access_key = config['MINIO_ROOT_USER']
secret_key = config['MINIO_ROOT_PASSWORD']

minio_client = MinioClient('localhost', 9000, access_key, secret_key)
csv_content_raw = minio_client.get_object(bucket_name='hist-data', obj_name='A.csv').data.decode('utf-8')
csv_reader = csv.reader(io.StringIO(csv_content_raw))
temp_row_holder = [line for line in csv_reader]
temp_row_holder
latest_row = temp_row_holder[-1]
with open("A.csv", 'w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerows(temp_row_holder)

In [15]:
cursor.execute(f"""
                SELECT info_id from public.fact_stock_data
                WHERE info_id = 'INFO_AAI';
                """)
result = cursor.fetchall()
result

[]

In [18]:
from datetime import datetime
current_time = int(datetime.now().timestamp())
print(current_time)


1722939147


In [22]:
import time
from datetime import timedelta
two_weeks_ago = datetime.now() - timedelta(weeks=2)
two_weeks_ago = int(two_weeks_ago.timestamp())
two_weeks_ago
int(time.time())

1722939448

In [27]:
from dags.module.object_client import MinioClient
from dotenv import dotenv_values
import csv
import pandas as pd

config = dotenv_values("dags/.env")
access_key = config['MINIO_ROOT_USER']
secret_key = config['MINIO_ROOT_PASSWORD']

minio_client = MinioClient("localhost", 9000, access_key, secret_key)
csv_content = minio_client.get_object("hist-data", "A.csv").data.decode('utf-8')
csv_content


'Date,Open,High,Low,Close,Adj Close,Volume\n1999-11-18,32.546494,35.765381,28.612303,31.473534,26.603022,62546380\n1999-11-19,30.713518,30.758226,28.478184,28.880545,24.411299,15234146\n1999-11-22,29.551144,31.473534,28.657009,31.473534,26.603022,6577870\n1999-11-23,30.400572,31.205294,28.612303,28.612303,24.184565,5975611\n1999-11-24,28.701717,29.998213,28.612303,29.372318,24.826965,4843231\n1999-11-26,29.238197,29.685265,29.148785,29.461731,24.902548,1729466\n1999-11-29,29.327612,30.355865,29.014664,30.132332,25.469374,4074751\n1999-11-30,30.042917,30.713518,29.282904,30.177038,25.507162,4310034\n1999-12-01,30.177038,31.071173,29.953505,30.713518,25.960619,2957329\n1999-12-02,31.294706,32.188843,30.892345,31.562946,26.678595,3069868\n1999-12-03,32.144135,32.680614,31.697067,31.831186,26.905327,3041629\n1999-12-06,32.367668,33.217094,32.322960,32.725323,27.661098,2250780\n1999-12-07,32.725323,32.904148,31.697067,32.367668,27.358799,2215970\n1999-12-08,32.367668,32.635906,32.054722,32.

In [20]:
for index, row in df.iterrows():
    # Process each row
    print(row['Date'], row["Open"])

1999-11-18 32.546494
1999-11-19 30.713518
1999-11-22 29.551144
1999-11-23 30.400572
1999-11-24 28.701717
1999-11-26 29.238197
1999-11-29 29.327612
1999-11-30 30.042917
1999-12-01 30.177038
1999-12-02 31.294706
1999-12-03 32.144135
1999-12-06 32.367668
1999-12-07 32.725323
1999-12-08 32.367668
1999-12-09 32.367668
1999-12-10 32.680614
1999-12-13 32.546494
1999-12-14 32.457081
1999-12-15 30.042917
1999-12-16 30.042917
1999-12-17 33.17239
1999-12-20 33.082977
1999-12-21 33.395924
1999-12-22 33.351215
1999-12-23 33.977112
1999-12-27 35.720673
1999-12-28 38.805435
1999-12-29 45.064377
1999-12-30 54.363377
1999-12-31 56.866955
2000-01-03 56.330471
2000-01-04 48.730328
2000-01-05 47.389126
2000-01-06 44.08083
2000-01-07 42.247852
2000-01-10 49.356224
2000-01-11 49.311516
2000-01-12 48.640915
2000-01-13 48.909157
2000-01-14 47.92561
2000-01-18 49.087982
2000-01-19 51.502148
2000-01-20 50.608009
2000-01-21 49.445637
2000-01-24 48.998569
2000-01-25 49.177395
2000-01-26 48.59621
2000-01-27 49.982